In [ ]:
# 卡尺工具

In [2]:
# 基础参数
import cv2
import math
import os
import numpy as np

img = cv2.imread('C:\\Users\\28104\\Desktop\\500.png',0)

CircleCenter = (1118.09, 1344.52)
CircleR = 682.449

CaliperNum = 36
CaliperX = 14
CaliperY = 80 # 像素大小

In [3]:
# img Tool
def show_img(img, name = 'img'):
    cv2.namedWindow(name, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_EXPANDED)
    cv2.imshow(name, img)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
def save_img(img, Img_name='img'):
    img_file = "C:\\Users\\28104\\Desktop\\pic"
    if not os.path.exists(img_file):
        os.mkdir(img_file)    
    cv2.imwrite(img_file + "\\" + Img_name + '.jpg', img)
    
green = (0, 255, 0)
red = (0, 0, 255)
yellow = (255, 0, 0)
# 灰度图
gray = 125

def draw_Pt(img1, Pt, text = ''):
    color = red
    wide = 2
    if isinstance(img1, str):
        global img
        img1 = img
    Pt = (round(Pt[0]), round(Pt[1]))
    
    cv2.circle(img1, Pt, 5, color, wide)
    if text != '':
        img1 = cv2.putText(img1, str(text), Pt, cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, wide)
    
    return img1

def draw_Cir(img1, CirR, Pt):
    color = red
    wide = 2
    if isinstance(img1, str):
        global img
        img1 = img
    Pt = (round(Pt[0]), round(Pt[1]))
    cv2.circle(img1, Pt, math.ceil(CirR), color, wide)
    
    return img
    
def draw_Rect(img1, Rect):  
    color = gray
    wide = 2
    
    # Pt = (round(Pt[0]), round(Pt[1]))  坐标四舍五入
    # math.ceil(CirR)                    半径向上取整
    Rect = [(round(pt[0]), round(pt[1])) for pt in Rect]

    if isinstance(img1, str):
        global img
        img1 = img
    if len(Rect) == 4:
        Pt1, Pt2, Pt4, Pt3 = Rect
        cv2.line(img1, Pt1, Pt2, color, wide)
        cv2.line(img1, Pt2, Pt3, color, wide)
        cv2.line(img1, Pt3, Pt4, color, wide)
        cv2.line(img1, Pt4, Pt1, color, wide)
    elif len(Rect) == 2:
        Pt1, Pt2 = Rect
        # cv2.rectangle(img, Pt1, Pt2, color, wide)
        cv2.line(img1, Pt1, Pt2, color, wide)
    return img1

In [7]:
class Caliper:
    #定义构造方法
    def __init__(self, img, theta, CaliperPt_Center, CaliperX = 6, CaliperY = 20):
        self.img = img
        self.CaliperX = CaliperX
        self.CaliperY = CaliperY
        self.theta = theta
        self.CaliperPt_Center = CaliperPt_Center
        self.CaliperPt_Edge = (self.CaliperPt_Center[0] + math.hypot(CaliperY/2, CaliperX/2) * math.cos(theta), self.CaliperPt_Center[1] + math.hypot(CaliperY/2, CaliperX/2) * math.sin(theta))
        #  self.ClpPixelMap 表示像素投影向量，self.ClpPixelFilter表示算子处理过的向量， self.EdgeCoordinate 求得的边缘点坐标，
        
         
    def getEdgePt(self, CircleCenter):
        # 得到卡尺边缘四个点的坐标
        CirR = math.hypot(self.CaliperY/2, self.CaliperX/2)
        CirPt = self.CaliperPt_Center
        line = (self.CaliperPt_Edge, CircleCenter)
        EdgePT = self.CaliperPt_Edge
        CentPt = CircleCenter

        CaliperPts = []
        if round(EdgePT[0]) == round(CentPt[0]):
            if abs(CirR) >= abs(EdgePT[0] - CirPt[0]):
                for evalItem in ['EdgePT[0] - CaliperX/2', 'EdgePT[0] + CaliperX/2']:
                    Pt1_x = eval(evalItem)
                    Pt2_x = eval(evalItem)
                    Pt1_y = CirPt[1] - math.sqrt(CirR**2 - (EdgePT[0]-CirPt[0])**2)
                    Pt2_y = CirPt[1] + math.sqrt(CirR**2 - (EdgePT[0]-CirPt[0])**2)
                    if abs(Pt1_y-EdgePT[1]) > abs(Pt2_y-EdgePT[1]):
                        Pt13 = (Pt2_x, Pt2_y)
                        Pt24 = (Pt1_x, Pt1_y)
                    else:
                        Pt13 = (Pt1_x, Pt1_y)
                        Pt24 = (Pt2_x, Pt2_y)
                    CaliperPts.extend([Pt13, Pt24])
            else:
                CaliperPts = []
        else:
            line_k = (EdgePT[1] - CentPt[1])/(EdgePT[0] - CentPt[0])
            for evalItem in ['EdgePT[1]-line_k*EdgePT[0]-CaliperX/(2*math.cos(theta))', 'EdgePT[1]-line_k*EdgePT[0]+CaliperX/(2*math.cos(theta))']:
                line_b = eval(evalItem)
                a = line_k**2 + 1
                b = 2*line_k*(line_b - CirPt[1]) - 2*CirPt[0]
                c = (line_b - CirPt[1])**2 + CirPt[0]**2 - CirR**2
                delta = b**2 - 4*a*c
                if delta >= 0:
                    Pt1_x = (-b - math.sqrt(delta))/(2*a)
                    Pt2_x = (-b + math.sqrt(delta))/(2*a)
                    Pt1_y = line_k*Pt1_x + line_b
                    Pt2_y = line_k*Pt2_x + line_b
                    if math.hypot(Pt1_x - EdgePT[0], Pt1_y - EdgePT[1]) > math.hypot(Pt2_x - EdgePT[0], Pt2_y - EdgePT[1]):
                        Pt13 = (Pt2_x, Pt2_y)
                        Pt24 = (Pt1_x, Pt1_y)
                    else:
                        Pt13 = (Pt1_x, Pt1_y)
                        Pt24 = (Pt2_x, Pt2_y)
                    CaliperPts.extend([Pt13, Pt24])
                else:
                    CaliperPts = []
                    break            
        return CaliperPts
    
    def CoordinatePixel(self, Coordinate):
        (x, y) = (Coordinate[0], Coordinate[1])
        (x1, y1) = (int(x), int(y))
        (x2, y2) = (x1+1, y1+1)
        pixel = self.img[x1][y1]*(x2-x)*(y2-y)+self.img[x2][y1]*(x-x1)*(y2-y)+self.img[x1][y2]*(x2-x)*(y-y1)+self.img[x2][y2]*(x-x1)*(y-y1)
        return round(pixel)
    
    def ClpPtCoordinates(self, CaliperPts):
        # 返回Y*X的列表, ClpPtCoordinates代表Caliper每一个点的坐标, ClpPtPixel代表每一个点的像素
        NumX = self.CaliperX
        NumY = self.CaliperY
        basePt = CaliperPts[0]
        distX = ((CaliperPts[2][0]-CaliperPts[0][0])/NumX, (CaliperPts[2][1]-CaliperPts[0][1])/NumX)
        distY = ((CaliperPts[1][0]-CaliperPts[0][0])/NumY, (CaliperPts[1][1]-CaliperPts[0][1])/NumY)
        self.ClpPtCoordinates = []
        ClpPtPixel = np.empty([NumY+1,NumX+1], dtype = int)
        for i in range((NumY+1)):
            list_X = []
            for j in range(NumX+1):
                Coordinate = (basePt[0]+distX[0]*j+distY[0]*i, basePt[1]+distX[1]*j+distY[1]*i)
                ClpPtPixel[i][j] = self.CoordinatePixel(Coordinate)
                list_X.append(Coordinate)
            self.ClpPtCoordinates.append(list_X)
        return self.ClpPtCoordinates, ClpPtPixel
    
    def ClpPixelConv(self, ClpPixelMap, FilterWidth = 1):
        length = len(ClpPixelMap)
        filters = [round(-1/FilterWidth, 2) for i in range(1, FilterWidth+1)] + [0] + [round(1/FilterWidth, 2) for i in range(1, FilterWidth+1)]
        filters = np.array(filters)
        ClpPixelFilter = np.zeros(length, dtype = np.float16)
        for i in range((length-FilterWidth*2)):
            ClpPixelFilter[i+FilterWidth] = np.sum(np.multiply(filters, ClpPixelMap[i: i+len(filters)])) 
        return ClpPixelFilter
    
    def ClpEdgePoint(self, ClpPtPixel):
        # 筛选边缘点，小于4的不要，选最大的三个点，取最小的那一个
        ClpPixelMap = np.apply_along_axis(np.mean,axis=1,arr=ClpPtPixel)
        self.ClpPixelMap = np.around(ClpPixelMap)
        self.ClpPixelFilter = self.ClpPixelConv(self.ClpPixelMap, 1)
        Edge  = self.getEdgeScore(self.ClpPixelFilter)

        if len(Edge)>0:
            EdgePtIdx = Edge[0]
            Posion = self.getEdge_posion(EdgePtIdx)
            print(EdgePtIdx, Posion)
            self.EdgeCoordinate = self.ClpPtCoordinates[EdgePtIdx][8]
        else:
            self.EdgeCoordinate = False
        return self.EdgeCoordinate
        
    def getEdge_posion(self, Edge):
        CaliperY = self.CaliperY
        return (Edge - CaliperY/2)
    
    def getEdge_point(self, pt_list):
        idx_pt_list = list(zip(range(len(pt_list)), pt_list))
        Item_max_idx, Item_max = max(idx_pt_list, key = lambda item : abs(item[1]))
        n = 0
        for i in idx_pt_list[Item_max_idx+1:]:
            if abs(Item_max)-abs(i[1])<3:
                n = n + 1
            else:
                break
        for i in idx_pt_list[Item_max_idx::-1]:
            if abs(Item_max)-abs(i[1])<3:
                n = n - 1
            else:
                break
        PtMax_idx = Item_max_idx + int((n+1)/2)
        return PtMax_idx, pt_list[PtMax_idx]


    def getEdgeScore(self, ClpPixelFilter):
        Pmax = abs(max(ClpPixelFilter.max(), ClpPixelFilter.min(), key = abs))
        edges = []
        predate = 0
        it = np.nditer(ClpPixelFilter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            # print(it[0], it.multi_index[0])
            if abs(it[0])<4:
                it[0] = 0
            if abs(it[0])> Pmax/2:
                if predate == 0:
                    edge = [[it.multi_index[0]], [float(it[0])]]
                    predate = 1
                elif len(edge)>0 and float(it[0]) * edge[1][-1]<0:
                    edges.append(edge)
                    edge = [[it.multi_index[0]], [float(it[0])]]
                else:
                    edge[0].append(it.multi_index[0])
                    edge[1].append(float(it[0]))
            else:
                if predate == 1:
                    edges.append(edge)
                    predate = 0
            it.iternext()

        if len(edges)>0:
            for i_edge in range(len(edges)):
                # print(edges[i_edge])
                idx, ptMax = self.getEdge_point(edges[i_edge][1])
                PtMax_idx = edges[i_edge][0][idx]
                
                edges[i_edge] = (PtMax_idx, ptMax)
                
            # 提取了多个可能的点，只取了其中最大的点
            edge = max(edges, key = lambda item : abs(item[1]))
        else:
            edge = []
        return edge

In [8]:
# search Caliper edge point
list_theta = [i * 2 * math.pi / CaliperNum for i in range(CaliperNum)]
Calipers_list = []
for theta in list_theta:
    CaliperPt_Center = (CircleCenter[0] + CircleR  * math.cos(theta), CircleCenter[1] + CircleR * math.sin(theta))
    # 实例化类并加入列表
    Caliper_obj = Caliper(img, theta, CaliperPt_Center, CaliperX, CaliperY)
    Calipers_list.append(Caliper_obj)
    CaliperPts = Caliper_obj.getEdgePt(CircleCenter)
    ClpPtCoordinates, ClpPtPixel = Caliper_obj.ClpPtCoordinates(CaliperPts)
    EdgeCoordinate = Caliper_obj.ClpEdgePoint(ClpPtPixel)
    if EdgeCoordinate:
#         print(EdgePtIdx)
#         EdgeCoordinate = ClpPtCoordinates[EdgePtIdx][8]
        Pt = (round(EdgeCoordinate[0]), round(EdgeCoordinate[1]))
        print(Pt)
        cv2.drawMarker(img, Pt, (255, 0, 0), markerType=1)
    
    if len(CaliperPts)==4:
        img = draw_Rect(img, CaliperPts)
        for pt in CaliperPts:
            img = draw_Pt(img, pt)
show_img(img)
save_img(img)

8 -32.0
(1833, 1346)
71 31.0
(1759, 1459)
57 17.0
(1743, 1573)
41 1.0
(1708, 1686)
68 28.0
(1619, 1766)
70 30.0
(1537, 1845)
73 33.0
(1442, 1907)
68 28.0
(1341, 1960)
33 -7.0
(881, 1992)
62 22.0
(787, 1916)
2 -38.0
(654, 1896)
52 12.0
(488, 1573)
57 17.0
(453, 1344)
54 14.0
(460, 1227)
2 -38.0
(441, 1097)
2 -38.0
(656, 792)
69 29.0
(792, 778)
48 8.0
(888, 710)
6 -34.0
(995, 639)
53 13.0
(1119, 675)
1 -39.0
(1244, 634)
23 -17.0
(1724, 996)
27 -13.0
(1772, 1108)
17 -23.0
(1813, 1223)


In [28]:
def ClpPixelConv(self, ClpPixelMap, FilterWidth = 1):
        length = len(ClpPixelMap)
        filters = [round(-1/FilterWidth, 2) for i in range(1, FilterWidth+1)] + [0] + [round(1/FilterWidth, 2) for i in range(1, FilterWidth+1)]
        filters = np.array(filters)
        padding = np.zeros(length, dtype = np.float16)
        for i in range((length-FilterWidth*2)):
            padding[i+FilterWidth] = np.sum(np.multiply(filters, ClpPixelMap[i: i+len(filters)])) 
        return padding

In [20]:
ClpPixelMap = np.apply_along_axis(np.mean,axis=1,arr=ClpPtPixel)
ClpPixelMap = np.around(ClpPixelMap)
padding = ClpPixelConv(ClpPixelMap, 2)

TypeError: object of type 'int' has no len()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(len(padding)), padding)
plt.show()
# 筛选边缘点，小于4的不要，选最大的三个点，取最小的那一个
MinPixel = 4
NumEdges = 3

padding

In [ ]:
# 废弃代码
# 圆和线段交点
def LineIntersectCircle(CirR, CirPt, line, CaliperX, theta):
# p is the circle parameter, lsp and lep is the two end of the line
    # CirPt is the circle center point, EdgePT and CentPt is the two end of the line
    EdgePT = line[0] # x1,y1
    CentPt = line[1] # x2,y2
    CirR             # r0 
    CirPt            # x0,y0 

    CaliperPts = []
    if round(EdgePT[0]) == round(CentPt[0]):
        if abs(CirR) >= abs(EdgePT[0] - CirPt[0]):
            for evalItem in ['EdgePT[0] - CaliperX/2', 'EdgePT[0] + CaliperX/2']:
                Pt1_x = eval(evalItem)
                Pt2_x = eval(evalItem)
                Pt1_y = CirPt[1] - math.sqrt(CirR**2 - (EdgePT[0]-CirPt[0])**2)
                Pt2_y = CirPt[1] + math.sqrt(CirR**2 - (EdgePT[0]-CirPt[0])**2)
                if abs(Pt1_y-EdgePT[1]) > abs(Pt2_y-EdgePT[1]):
                    Pt13 = (Pt2_x, Pt2_y)
                    Pt24 = (Pt1_x, Pt1_y)
                else:
                    Pt13 = (Pt1_x, Pt1_y)
                    Pt24 = (Pt2_x, Pt2_y)
                CaliperPts.extend([Pt13, Pt24])  
        else:
            CaliperPts = []
    else:
        line_k = (EdgePT[1] - CentPt[1])/(EdgePT[0] - CentPt[0])
        for evalItem in ['EdgePT[1]-line_k*EdgePT[0]-CaliperX/(2*math.cos(theta))', 'EdgePT[1]-line_k*EdgePT[0]+CaliperX/(2*math.cos(theta))']:
            line_b = eval(evalItem)
            a = line_k**2 + 1
            b = 2*line_k*(line_b - CirPt[1]) - 2*CirPt[0]
            c = (line_b - CirPt[1])**2 + CirPt[0]**2 - CirR**2
            delta = b**2 - 4*a*c
            if delta >= 0:
                Pt1_x = (-b - math.sqrt(delta))/(2*a)
                Pt2_x = (-b + math.sqrt(delta))/(2*a)
                Pt1_y = line_k*Pt1_x + line_b
                Pt2_y = line_k*Pt2_x + line_b
                if math.hypot(Pt1_x - EdgePT[0], Pt1_y - EdgePT[1]) > math.hypot(Pt2_x - EdgePT[0], Pt2_y - EdgePT[1]):
                    Pt13 = (Pt2_x, Pt2_y)
                    Pt24 = (Pt1_x, Pt1_y)
                else:
                    Pt13 = (Pt1_x, Pt1_y)
                    Pt24 = (Pt2_x, Pt2_y)
                CaliperPts.extend([Pt13, Pt24])
            else:
                CaliperPts = [] 
                break
    return CaliperPts